# APIM ❤️ AI Agents

## Agent 2 Agent Communication over MCP for MCP-enabled agents lab (initial release)
![flow](../../images/mcp-agent-2-agent.gif)

Playground to experiment with [Agent 2 Agent Communication over MCP](https://devblogs.microsoft.com/blog/can-you-build-agent2agent-communication-on-mcp-yes) agents with which in turn use [MCP](https://modelcontextprotocol.io/) for tools with Azure API Management to enable plug & play of tools to LLMs. 

This lab relies on tools provided and deployed as part of shared infrastructure:
- Basic oncall service: provides a tool to get a list of random people currently on-call with their status and time zone.
- Basic weather service: provide tools to get cities for a given country and retrieve random weather information for a specified city.

MCP-enabled agents are then deployed within ACA (Azure Container Apps) as MCP servers.

This lab demonstrates the art of the possible of creating hetrogenous multi-agentic system with agents created using multiple orchestrators, and then allowing a single unifying protocol to communicate accross the through APIM for Authn/Authz

### Prerequisites

- [Python 3.13 or later](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

**Important** This lab follows a different architecture from other lab.

Please deploy the shared infrastructure from this [Notebook](deploy-a2a-infra-assests.ipynb) before proceeding

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...


In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"

apim_sku = 'Basicv2'

openai_resources = [ {"name": "openai1", "location": "uksouth"}]
openai_model_name = "gpt-4o-mini"
openai_model_version = "2024-07-18"
openai_model_sku = "GlobalStandard"
openai_deployment_name = "gpt-4o-mini"
openai_api_version = "2024-10-21"

build = 0
weather_mcp_server_image = "weather-mcp-server"
weather_mcp_server_src = "src/weather/mcp-server"

oncall_mcp_server_image = "oncall-mcp-server"
oncall_mcp_server_src = "src/oncall/mcp-server"

utils.print_ok('Notebook initialized')

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM Gateway URL')
    apim_resource_name = utils.get_deployment_output(output, 'apimResourceName', 'APIM Resource Name')
    apim_subscription_key = utils.get_deployment_output(output, 'apimSubscriptionKey', 'APIM Subscription Key (masked)', True)
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')
    container_registry_name = utils.get_deployment_output(output, 'containerRegistryName', 'Container Registry Name')
    weather_containerapp_resource_name = utils.get_deployment_output(output, 'weatherMCPServerContainerAppResourceName', 'Weather Container App Resource Name')
    oncall_containerapp_resource_name = utils.get_deployment_output(output, 'oncallMCPServerContainerAppResourceName', 'Oncall Container App Resource Name')

    a2a_weather_containerapp_resource_name = utils.get_deployment_output(output, 'a2AWeatherAgentServerContainerAppResourceName', 'A2A (Weather) Agent Container App Resource Name')
    a2a_oncall_containerapp_resource_name = utils.get_deployment_output(output, 'a2AOncallAgentServerContainerAppResourceName', 'A2A (Oncall) Agent Container App Resource Name')

    a2a_weather_a2a_agent_ep = utils.get_deployment_output(output, 'a2AWeatherAgentServerContainerAppFQDN', 'A2A (Weather) Agent Endpoint')
    a2a_oncall_a2a_agent_ep = utils.get_deployment_output(output, 'a2AOncallAgentServerContainerAppFQDN', 'A2A (Oncall) Agent Endpoint')

inference_api_path = ""
inference_api_version = "2025-03-01-preview"


### Building Customisable MCP-Server with MCP-enabled Agent images for deployment in ACA

#### Semantic-Kernel

In [ ]:
build = build + 1

mcp_sk_server_image = "sk-agent-mcp-server"
mcp_sk_server_src = "src/mcp_sk_servers/src"

utils.run(f"az acr build --image {mcp_sk_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {mcp_sk_server_src}/Dockerfile {mcp_sk_server_src}/. --no-logs", 
          "Generic MCP SK Server image was successfully built", "Failed to build the Generic MCP SK Server image")

In [ ]:
title="Weather"
mcp_url = "/weather"

utils.run(f'az containerapp secret set -n  {a2a_weather_containerapp_resource_name} -g {resource_group_name} --secrets apimsubscriptionkey={apim_subscription_key}', "Weather A2A Server secret updated", "Weather A2A Server secret update failed")
utils.run(f'az containerapp update -n {a2a_weather_containerapp_resource_name} -g  {resource_group_name} --image "{container_registry_name}.azurecr.io/{mcp_sk_server_image}:v0.{build}" --set-env-vars TITLE={title} MCP_URL={mcp_url} APIM_GATEWAY_URL={apim_resource_gateway_url} OPENAI_API_VERSION={openai_api_version} OPENAI_DEPLOYMENT_NAME={openai_deployment_name} APIM_SUBSCRIPTION_KEY=secretref:apimsubscriptionkey', 
          "Weather MCP SK Server with MCP deployment succeeded", "Weather MCP SK Server with MCP deployment failed")

title="Oncall"
mcp_url = "/oncall"

utils.run(f'az containerapp secret set -n  {a2a_oncall_containerapp_resource_name} -g {resource_group_name} --secrets apimsubscriptionkey={apim_subscription_key}', "Oncall A2A Server secret updated", "Oncall A2A Server secret update failed")
utils.run(f'az containerapp update -n {a2a_oncall_containerapp_resource_name} -g  {resource_group_name} --image "{container_registry_name}.azurecr.io/{mcp_sk_server_image}:v0.{build}" --set-env-vars TITLE={title} MCP_URL={mcp_url} APIM_GATEWAY_URL={apim_resource_gateway_url} OPENAI_API_VERSION={openai_api_version} OPENAI_DEPLOYMENT_NAME={openai_deployment_name} APIM_SUBSCRIPTION_KEY=secretref:apimsubscriptionkey', 
          "Oncall MCP SK Server with MCP deployment succeeded", "Oncall MCP SK Server with MCP deployment failed")

In [ ]:
utils.print_info(f"MCP URL for Weather Agent direct to ACA: https://{a2a_weather_a2a_agent_ep}/mcp")
utils.print_info(f"MCP URL for OnCall Agent direct to ACA: https://{a2a_oncall_a2a_agent_ep}/mcp")

utils.print_info(f'MCP URL for Weather Agent via APIM: {apim_resource_gateway_url}/weather-agent-mcp/mcp')
utils.print_info(f'MCP URL for OnCall Agent via APIM: {apim_resource_gateway_url}/oncall-agent-mcp/mcp')

### AG Agent of MCP-Agents

In [ ]:
import asyncio
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.tools.mcp import StreamableHttpServerParams, StreamableHttpMcpToolAdapter, mcp_server_tools
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console

async def run_agent(prompt: str) -> None:
    # Create server params for the remote MCP service
    weather_agent_mcp_params = StreamableHttpServerParams(
        url=f"{apim_resource_gateway_url}/weather-agent-mcp/mcp",
        headers={"Content-Type": "application/json"},
        timeout=30,  # Connection timeout in seconds
    )
    weather_tools = await mcp_server_tools(weather_agent_mcp_params)

    oncall_agent_mcp_params = StreamableHttpServerParams(
        url=f"{apim_resource_gateway_url}/oncall-agent-mcp/mcp",
        headers={"Content-Type": "application/json"},
        timeout=30,  # Connection timeout in seconds
    )

    # Get all available tools
    oncall_tools = await mcp_server_tools(oncall_agent_mcp_params)

    # Create an agent that can use the translation tool
    model_client = AzureOpenAIChatCompletionClient(azure_deployment=openai_model_name, model=openai_model_name,
                azure_endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
                api_key=apim_subscription_key,
                api_version=inference_api_version
    )
    agent = AssistantAgent(
        name="weather",
        model_client=model_client,
        reflect_on_tool_use=True,
        tools=weather_tools + oncall_tools,  # type: ignore
        system_message="You are a helpful assistant.",
    )
    await Console(
        agent.run_stream(task=prompt)
    )

import nest_asyncio
nest_asyncio.apply()

asyncio.run(run_agent("What's the weather in Lisbon, Cairo and London? and who's oncall in CET only?"))


### SK Agent of MCP-Agents

In [ ]:
import asyncio
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin

user_input = "What's the current weather in Lisbon and London? and who's oncall today in CET only?"

weather_agent = MCPStreamableHttpPlugin(
    name="Weather",
    url=f"{apim_resource_gateway_url}/weather-agent-mcp/mcp",
    description="Weather Agent",
    )

oncall_agent = MCPStreamableHttpPlugin(
    name="OnCall",
    url=f"{apim_resource_gateway_url}/oncall-agent-mcp/mcp",
    description="OnCall Agent",
)

await weather_agent.connect()  # Ensure the plugin is connected before using it
await oncall_agent.connect()  # Ensure the plugin is connected before using it

agent = ChatCompletionAgent(
    service=AzureChatCompletion(
        endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
        api_key=apim_subscription_key,
        api_version=inference_api_version,                
        deployment_name=openai_model_name  # Use the first model from the models_config
    ),
    name="IssueAgent",
    instructions="Answer questions using the tools available to you.",
    plugins=[weather_agent, oncall_agent],
)

thread: ChatHistoryAgentThread | None = None

print(f"# User: {user_input}")
# 2. Invoke the agent for a response
response = await agent.get_response(messages=user_input, thread=thread)
print(f"# {response.name}: {response} ")
thread = response.thread # type: ignore

# 3. Cleanup: Clear the thread
await thread.delete() if thread else None
# await weather_plugin.close()  # Ensure the plugin is disconnected after use
# await oncall_plugin.close()  # Ensure the plugin is disconnected after use
